In [1]:
# Generate the list of index files archived in EDGAR since start_year (earliest: 1993) until the most recent quarter
import datetime

current_year = datetime.date.today().year
current_quarter = (datetime.date.today().month - 1) // 3 + 1
start_year = 2021
years = list(range(start_year, current_year))
quarters = ['QTR1', 'QTR2', 'QTR3', 'QTR4']
history = [(y, q) for y in years for q in quarters]
for i in range(1, current_quarter + 1):
    history.append((current_year, 'QTR%d' % i))
urls = ['https://www.sec.gov/Archives/edgar/full-index/%d/%s/crawler.idx' % (x[0], x[1]) for x in history]
urls.sort()
print(urls)


['https://www.sec.gov/Archives/edgar/full-index/2021/QTR1/crawler.idx', 'https://www.sec.gov/Archives/edgar/full-index/2021/QTR2/crawler.idx', 'https://www.sec.gov/Archives/edgar/full-index/2021/QTR3/crawler.idx', 'https://www.sec.gov/Archives/edgar/full-index/2021/QTR4/crawler.idx', 'https://www.sec.gov/Archives/edgar/full-index/2022/QTR1/crawler.idx', 'https://www.sec.gov/Archives/edgar/full-index/2022/QTR2/crawler.idx']


In [46]:
# # Download index files and write content into SQLite
# import sqlite3
# import requests
# import pandas as pd
# con = sqlite3.connect('edgar_htm_idx.db')
# cur = con.cursor()
# cur.execute('DROP TABLE IF EXISTS idx')
# cur.execute('CREATE TABLE idx (conm TEXT, type TEXT, cik TEXT, date TEXT, path TEXT)')


In [2]:

# Download index files and write content into SQLite
import sqlite3
import requests
import pandas as pd
con = sqlite3.connect('edgar_htm_idx.db')
cur = con.cursor()
cur.execute('DROP TABLE IF EXISTS idx')
cur.execute('CREATE TABLE idx (conm TEXT, type TEXT, cik TEXT, date TEXT, path TEXT)')


# for url in urls:
#     lines = requests.get(url).text.splitlines()
#     nameloc = lines[7].find('Company Name')
#     typeloc = lines[7].find('Form Type')
#     cikloc = lines[7].find('CIK')
#     dateloc = lines[7].find('Date Filed')
#     urlloc = lines[7].find('URL')
#     records = [tuple([line[:typeloc].strip(), line[typeloc:cikloc].strip(), line[cikloc:dateloc].strip(),
#                       line[dateloc:urlloc].strip(), line[urlloc:].strip()]) for line in lines[9:]]
#     cur.executemany('INSERT INTO idx VALUES (?, ?, ?, ?, ?)', records)
#     print(url, 'downloaded and wrote to SQLite')

# con.commit()
# con.close()

for url in urls:
    lines = requests.get(url, allow_redirects=True, headers={"User-Agent": "Mozilla/5.0"}).text.splitlines()
    print("Downloading from : =>      "+ url)
    nameloc = lines[7].find('Company Name')
    typeloc = lines[7].find('Form Type')
    cikloc = lines[7].find('CIK')
    dateloc = lines[7].find('Date Filed')
    urlloc = lines[7].find('URL')
    records = [tuple([line[:typeloc].strip(), line[typeloc:cikloc].strip(), line[cikloc:dateloc].strip(),
                      line[dateloc:urlloc].strip(), line[urlloc:].strip()]) for line in lines[10:]]
    cur.executemany('INSERT INTO idx VALUES (?, ?, ?, ?, ?)', records)
    print(url, 'downloaded and wrote to SQLite')

con.commit()
con.close()


# Write SQLite database to Stata


https://www.sec.gov/Archives/edgar/full-index/2021/QTR1/crawler.idx downloaded and wrote to SQLite
https://www.sec.gov/Archives/edgar/full-index/2021/QTR2/crawler.idx downloaded and wrote to SQLite
https://www.sec.gov/Archives/edgar/full-index/2021/QTR3/crawler.idx downloaded and wrote to SQLite
https://www.sec.gov/Archives/edgar/full-index/2021/QTR4/crawler.idx downloaded and wrote to SQLite
https://www.sec.gov/Archives/edgar/full-index/2022/QTR1/crawler.idx downloaded and wrote to SQLite
https://www.sec.gov/Archives/edgar/full-index/2022/QTR2/crawler.idx downloaded and wrote to SQLite


In [3]:
import pandas
from sqlalchemy import create_engine


engine = create_engine('sqlite:///edgar_htm_idx.db')
with engine.connect() as conn, conn.begin():
    data = pandas.read_sql_table('idx', conn)
    print(data)
    # ta = pd.DataFrame([sub.split("|") for sub in  data])
    data.to_csv('data.csv',sep = ",")
    # data.to_txt('edgar_htm_idx.csv')

                                                      conm  type      cik  \
0                                    01 Carbon Capture, LP     D  1844686   
1         0X Fund I, a series of Kings County Ventures, LP     D  1843213   
2                                    1 800 FLOWERS COM INC  10-Q  1084869   
3                                    1 800 FLOWERS COM INC     4  1084869   
4                                    1 800 FLOWERS COM INC     4  1084869   
...                                                    ...   ...      ...   
1769614                                  williams craig a.     4  1848752   
1769615                           xCraft Enterprises, Inc.   1-K  1670450   
1769616                           xCraft Enterprises, Inc.  C-AR  1670450   
1769617                           xCraft Enterprises, Inc.   C/A  1670450   
1769618  yourspace, a Series of GoodPaper Ventures Fund...     D  1913076   

               date                                               path  
0 

In [4]:
import csv
import random
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager


 
with open('log.csv', 'w', newline='') as log:
    logwriter = csv.writer(log)
 
    with open('data.csv', newline='') as infile:
        records = csv.reader(infile)

        for r in records:
            print(len(r))
            log_row = r.copy()
            print('Start fetching URL to', r[1], r[2], 'filed on', r[3], '...')
            start_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())
 
            driver = webdriver.Chrome(ChromeDriverManager().install())
 
            try:
                driver.get(r[5])
                time.sleep(3 + random.random() * 3)
                filing_date = driver.find_element_by_xpath('//*[@id="formDiv"]/div[2]/div[1]/div[2]').text
                period_of_report = driver.find_element_by_xpath('//*[@id="formDiv"]/div[2]/div[2]/div[2]').text
                form_text = driver.find_element_by_xpath('//*[@id="formDiv"]/div/table/tbody/tr[2]/td[3]/a').text
                form_link = driver.find_element_by_link_text(form_text).get_attribute('href')
                end_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())
                print('Success!', start_time, ' --> ', end_time, '\n')
                log_row = log_row + [start_time, end_time, filing_date, period_of_report, form_link]
 
            except:
                end_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())
                print('Error!', start_time, ' --> ', end_time, '\n')
                log_row = log_row + [start_time, end_time, 'ERROR!']
 
            driver.quit()
 
            logwriter.writerow(log_row)

6
Start fetching URL to conm type filed on cik ...



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_mac64_m1.zip
[WDM] - Driver has been saved in cache [/Users/harryzhang/.wdm/drivers/chromedriver/mac64_m1/102.0.5005.61]
/var/folders/z5/102n99n926x7011cmbz3_q7w0000gn/T/ipykernel_55630/1026907867.py:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Error! 2022-06-15 17:46:26  -->  2022-06-15 17:46:30 

6
Start fetching URL to 01 Carbon Capture, LP D filed on 1844686 ...



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/harryzhang/.wdm/drivers/chromedriver/mac64_m1/102.0.5005.61/chromedriver] found in cache


Success! 2022-06-15 17:46:30  -->  2022-06-15 17:46:37 

6
Start fetching URL to 0X Fund I, a series of Kings County Ventures, LP D filed on 1843213 ...



...

Error! 2022-06-15 17:46:47  -->  2022-06-15 17:46:51 

6
Start fetching URL to 1 800 FLOWERS COM INC 4 filed on 1084869 ...



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/harryzhang/.wdm/drivers/chromedriver/mac64_m1/102.0.5005.61/chromedriver] found in cache


Error! 2022-06-15 17:46:52  -->  2022-06-15 17:46:57 

6
Start fetching URL to 1 800 FLOWERS COM INC 4 filed on 1084869 ...



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome


KeyboardInterrupt: 

In [11]:
for url in urls:
    lines = requests.get(url, allow_redirects=True, headers={"User-Agent": "Mozilla/5.0"}).text.splitlines()
    ta = pd.DataFrame([sub.split("|") for sub in byte_data])
    ta.to_csv("data.csv",sep = ',')

In [58]:
engine = create_engine('sqlite:///edgar_htm_idx.db')
with engine.connect() as conn, conn.begin():
    data = pandas.read_sql_table('idx', conn)
    data.to_csv(f'{(x[0], x[1]) for x in history}.csv',sep = "|")

In [17]:
def save(data):
    with open('edgar_htm_idx.csv.csv', 'a', encoding='utf-8', newline="")as f:
        f_csv = csv.writer(f, delimiter='｜', quotechar='\"')
        print(type(data))
        f_csv.writerows([data])

In [8]:
database_name = "edgar_htm_idx.dta"
database = pd.read_stata(database_name)
print("There are a total of {} entries of filing links".format(str(len(database))))
print("Note: these are not document links, but filing links.")

# dropping the useless 'index' column
database.drop(columns = ['index'],inplace=True)
def select_queries(user_requests):
    # to be implemented
    return 

# output what you want to query.csv

print("Try 5 first before the entire database")
print("Depending on the conditions, downloading 1 filing can take up to 30 seconds")



database.head(5).to_csv("query.csv")

There are a total of 2810866 entries of filing links
Note: these are not document links, but filing links.
Try 5 first before the entire database
Depending on the conditions, downloading 1 filing can take up to 30 seconds


In [8]:
import csv
byte_data = requests.get('https://www.sec.gov/Archives/edgar/full-index/2020/QTR1/crawler.idx', allow_redirects=True, headers={"User-Agent": "XYZ/3.0"}).text.splitlines()[11:]


In [10]:
ta = pd.DataFrame([sub.split("|") for sub in byte_data])
ta.to_csv("data.csv",sep = ',',index=None)

In [48]:
print()
print()
import pandas as pd

database_name = "edgar_htm_idx.dta"
database = pd.read_stata(database_name)
print("There are a total of {} entries of filing links".format(str(len(database))))
print("Note: these are not document links, but filing links.")

# dropping the useless 'index' column
database.drop(columns = ['index'],inplace=True)
def select_queries(user_requests):
    # to be implemented
    return 

# output what you want to query.csv

print("Try 5 first before the entire database")
print("Depending on the conditions, downloading 1 filing can take up to 30 seconds")



database.head(5).to_csv("query.csv")  # choose how many to download



There are a total of 440 entries of filing links
Note: these are not document links, but filing links.
Try 5 first before the entire database
Depending on the conditions, downloading 1 filing can take up to 30 seconds


In [22]:

import csv
import random
import time

from selenium import webdriver

with open('log.csv', 'w', newline='') as log:
    logwriter = csv.writer(log)

    with open('sample.csv', newline='') as infile:
        records = csv.reader(infile)

        for r in records:
            log_row = r.copy()
            print('Start fetching URL to', r[2], r[3], 'filed on', r[4], '...')
            start_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())

            driver = webdriver.Chrome('/Applications/Google Chrome Beta.app')

            try:
                driver.get(r[5])
                time.sleep(3 + random.random() * 3)
                filing_date = driver.find_element_by_xpath('//*[@id="formDiv"]/div[2]/div[1]/div[2]').text
                period_of_report = driver.find_element_by_xpath('//*[@id="formDiv"]/div[2]/div[2]/div[2]').text
                form_text = driver.find_element_by_xpath('//*[@id="formDiv"]/div/table/tbody/tr[2]/td[3]/a').text
                form_link = driver.find_element_by_link_text(form_text).get_attribute('href')
                end_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())
                print('Success!', start_time, ' --> ', end_time, '\n')
                log_row = log_row + [start_time, end_time, filing_date, period_of_report, form_link]

            except:
                end_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())
                print('Error!', start_time, ' --> ', end_time, '\n')
                log_row = log_row + [start_time, end_time, 'ERROR!']

            driver.quit()

            logwriter.writerow(log_row)

Start fetching URL to cik filing date filed on index url ...


/var/folders/z5/102n99n926x7011cmbz3_q7w0000gn/T/ipykernel_11205/348738147.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Applications/Google Chrome Beta.app')


WebDriverException: Message: 'Google Chrome Beta.app' executable may have wrong permissions. Please see https://chromedriver.chromium.org/home


In [23]:
import requests
import os
import time
import sys

# List of years to be searched
years = [2000 + x for x in range(10,18)]

# List of quarters to be searched
quarters = ['QTR1', 'QTR2', 'QTR3', 'QTR4']

# The absolute path to the folder for saving the index files

base_path = str('./')
sys.path.append(base_path)
# Get a list of the folders in the base path.
current_dirs = os.listdir(path=base_path)

# Loop over the years and quarters. For each year/quarter combination, get the corresponding
# index file from EDGAR, and save it to the local machine as a text file.
for yr in years:

    # Check current_dirs for a folder labeled 'yr' (i.e. 2009, 2015, etc)
    # If the folder does not exist, then create it.
    if str(yr) not in current_dirs:
        os.mkdir('/'.join([base_path, str(yr)]))

    # Get a list of the files inside the year folder.
    current_files = os.listdir('/'.join([base_path, str(yr)]))

    for qtr in quarters:
        # Use the following filename pattern to store the index files locally.
        local_filename = f'xbrl-index-{yr}-{qtr}.txt'

        # Create the absolute path for storing the index file. (e.g.
        # C:/Users/Martin/edgar/indexes/2015/xbrl-index-2015-QTR3.txt)
        local_file_path = '/'.join([base_path, str(yr), local_filename])

        # Check to see if the index file for the current year/quarter combination is already
        # saved locally. If it is, then skip to the next year/quarter combination.
        if local_filename in current_files:
            print(f'Skipping index file for {yr}, {qtr} because it is already saved.')
            continue

        # Define the url at which to get the index file.
        url = f'https://www.sec.gov/Archives/edgar/daily-index/{yr}/{qtr}/crawler.idx'

        # Get the index file from EDGAR and save it to a text file. Note that to save a file
        # rather than bringing it into memory, set stream=True and use r.iter_content()
        # to break the incoming stream into chunks (here arbitrarily of size 10240 bytes)
        headers = { 'User-Agent': 'Mozilla/5.0', }
        r = requests.get(url, stream=True)
        with open(local_file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=10240):
                f.write(chunk)


        r = requests.get(url, stream=True, headers = headers)
        # Wait one-tenth of a second before sending another request to EDGAR.
        time.sleep(10)